In [22]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from math import radians, degrees, cos, sin, asin, sqrt

from tqdm import tqdm_notebook as tqdm

import feather


In [23]:
se = feather.read_dataframe("../../data_files/B/once/75days/stop_events_with_geo_train_test_averages_prev_next_dwell.feather",
                            columns=["index",
                                     'line_distance','workid',
                                     'publicName',
                                     'segment_name'])
se = se.set_index(se.columns[0])

In [48]:
routes = pd.read_csv("../../Bournemouth GTFS/routes.csv")
shapes = pd.read_csv("../../Bournemouth GTFS/shapes.csv")
stops = pd.read_csv("../../Bournemouth GTFS/stops.txt")
trips = pd.read_csv("../../Bournemouth GTFS/trips.txt")

### Plan
- Load se and get all unique segment_name and public name combinations. 
- For each public name find the shape(s) from routes -> trips. 
   - If you can do better with workID or something then even better
- From the shape(s) find the closest dots on the shape 
   - (this might not be as good as finding a midpoint between two shapes)
- Once we have a start and end point find the length of all the lines between them and add them up

In [24]:
to_find = se.groupby(['segment_name', 'publicName']).first()

In [38]:
to_find['route_id'] = routes.set_index('route_short_name').loc[to_find.index.get_level_values(1)]['route_id'].values

/Users/tommelamed/anaconda3/envs/busses/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  """Entry point for launching an IPython kernel.


In [66]:
trips_to_shapes_dict = {}

for route_id, route_group in trips.groupby('route_id'):
    matching_shapes = route_group.groupby('shape_id').first().index.values
    
    trips_to_shapes_dict[route_id] = matching_shapes


In [70]:
class point():
    
    def __init__(self, lat, lon):
        self.lat = lat
        self.lon = lon

In [67]:
def haversine_np(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points
    on the earth (specified in decimal degrees)

    All args must be of equal length.    

    """
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = np.sin(dlat/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2.0)**2

    c = 2 * np.arcsin(np.sqrt(a))
    km = 6367 * c
    return km

In [72]:
def nearest_point(stop_point, shape):
    distances = haversine_np([stop_point.lon], [stop_point.lat], shape[:,0], shape[:,1])
    
    return shape[np.argsort(distances)]

In [73]:
to_find.iloc[0]

workid                167_101_10
line_distance            0.60914
route_id         DOAO01a:DOAO01a
Name: (1200BOB20136_1200BOB20141, 1a), dtype: object

In [74]:
stops

,stop_id,stop_code,stop_name,stop_desc,stop_lat,stop_lon,zone_id,stop_url,location_type,parent_station,stop_timezone,wheelchair_boarding
0,1280POA11309,NaN,Abbotsbury Road,NaN,50.763061,-2.003464,NaN,NaN,NaN,NaN,NaN,NaN
1,1290DOB20933,NaN,Acton Road,NaN,50.750471,-1.907032,NaN,NaN,NaN,NaN,NaN,NaN
2,1290DOB20934,NaN,Acton Road,NaN,50.750696,-1.906960,NaN,NaN,NaN,NaN,NaN,NaN
3,1280POA11494,NaN,Ad Astra Infant School,NaN,50.749648,-1.964114,NaN,NaN,NaN,NaN,NaN,NaN
4,1280POA11493,NaN,Ad Astra Infant School,NaN,50.749262,-1.964639,NaN,NaN,NaN,NaN,NaN,NaN
5,1200DOA05374,NaN,Adventure Wonderland,NaN,50.776529,-1.845299,NaN,NaN,NaN,NaN,NaN,NaN
6,1200DOA05373,NaN,Adventure Wonderland,NaN,50.776403,-1.845356,NaN,NaN,NaN,NaN,NaN,NaN
7,1200DOA91109,NaN,AIM Building,NaN,50.789390,-1.846392,NaN,NaN,NaN,NaN,NaN,NaN
8,1200BOB20154,NaN,Airfield Way,NaN,50.739133,-1.749326,NaN,NaN,NaN,NaN,NaN,NaN
9,1200BOB20155,NaN,Airfield Way,NaN,50.738846,-1.749625,NaN,NaN,NaN,NaN,NaN,NaN
